In [5]:
!pip install pandas
!pip install folium
!pip install HeatMap


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\Comunicaciones 6\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\Comunicaciones 6\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


  Using cached heatmap-2.2.1.zip (25 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [25 lines of output]
      Traceback (most recent call last):
        File "C:\Users\Comunicaciones 6\AppData\Local\Programs\Python\Python313\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
          ~~~~^^
        File "C:\Users\Comunicaciones 6\AppData\Local\Programs\Python\Python313\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ~~~~^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\Comunicaciones 6\AppData\Local\Programs\Python\Python313\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 118, in get_requires_for_build_wheel
          return hook(config_settings)
        File "C:\Users\Comunicaciones 6\AppData\Local\Temp\p

In [6]:
import pandas as pd
import folium
from folium.plugins import HeatMap

In [7]:
def obtener_icono(tipo_zona, tamaño='32'):
    # Diccionario con URLs de los íconos en icon-icons.com, donde el tamaño puede ser ajustado
    iconos = {
        'Puentes': f"https://cdn.icon-icons.com/icons2/995/PNG/{tamaño}/Bridge_Constructor_icon-icons.com_75321.png",
        'Antenas Telefónicas': f"https://cdn.icon-icons.com/icons2/1829/PNG/{tamaño}/antennasignalobservatoryradio-115839_115790.png",
        'Placa Huella': f"https://cdn.icon-icons.com/icons2/3601/PNG/{tamaño}/journey_road_street_asph_highway_icon_226304.png",
        'Escuelas': f"https://cdn.icon-icons.com/icons2/2313/PNG/{tamaño}/teacher_education_school_lecture_student_icon_141984.png",
    }

    # Si no se encuentra el tipo de zona, se usará un ícono por defecto
    icono_url = iconos.get(tipo_zona, f'https://cdn.icon-icons.com/icons2/3601/PNG/{tamaño}/default_icon.png')

    return icono_url

In [8]:
# Función principal para generar el mapa
def generar_mapa(factor_input, df_datos, municipios_santander):
    # Validar si el factor ingresado existe en los datos
    if factor_input not in df_datos['Factor o Carencia'].unique():
        print(f"El factor '{factor_input}' no existe en los datos. Intenta con otro factor.")
        return
    
    # Filtrar las 5 zonas más afectadas por el factor
    columna_factor = 'Unidades'  # La columna que contiene los valores numéricos para el análisis
    zonas_afectadas = df_datos[df_datos['Factor o Carencia'] == factor_input].nlargest(5, columna_factor)
    
    # Crear el mapa centrado en Santander
    mapa_santander = folium.Map(
        location=[7.13, -73.13], 
        zoom_start=10
    )
    
    # Crear una lista de coordenadas (lat, lon) y valores para el heatmap
    heat_data = []
    
    # Añadir marcadores y puntos para el heatmap
    for _, fila in zonas_afectadas.iterrows():
        nombre = fila['Ubicación']  # Usar la columna 'Ubicación' para el nombre de la zona
        
        # Verificar si el municipio existe en el diccionario de coordenadas
        if nombre in municipios_santander:
            latitud, longitud = municipios_santander[nombre]  # Obtener latitud y longitud del municipio
        else:
            print(f"Advertencia: El municipio '{nombre}' no tiene coordenadas definidas. Saltando...")
            continue  # Si no existe, saltamos este municipio
        
        tipo_zona = fila.get('Actores', 'Otro')  # Usar un valor por defecto si no existe
        factor_value = fila[columna_factor]  # Obtener el valor correspondiente de 'Unidades'
        
        # Obtener el ícono correspondiente
        icon_url = obtener_icono(tipo_zona)  # Llamar a la función para obtener el ícono
        
        # Crear texto detallado para el marcador
        popup_texto = f"""
        <strong>{nombre}</strong><br>
        Latitud: {latitud}<br>
        Longitud: {longitud}<br>
        {factor_input.capitalize()}: {factor_value}<br>
        Tipo de Zona: {tipo_zona}
        """
        
        # Agregar marcador al mapa
        folium.Marker(
            location=[latitud, longitud],
            popup=folium.Popup(popup_texto, max_width=300),
            icon=folium.CustomIcon(icon_url, icon_size=(30, 30))  # Usar ícono local o URL
        ).add_to(mapa_santander)

        # Añadir al heatmap (lat, lon, valor)
        heat_data.append([latitud, longitud, factor_value])
    
    # Añadir el heatmap al mapa
    HeatMap(heat_data).add_to(mapa_santander)
    
    # Agregar capas base
    folium.TileLayer("OpenStreetMap", name="Mapa Normal").add_to(mapa_santander)
    folium.TileLayer(
        tiles="http://{s}.tile.stamen.com/terrain/{z}/{x}/{y}.jpg",
        attr="Map tiles by Stamen Design, CC BY 3.0 — Map data © OpenStreetMap contributors",
        name="Mapa Terreno"
    ).add_to(mapa_santander)
    folium.TileLayer(
        tiles="http://{s}.tile.stamen.com/toner/{z}/{x}/{y}.png",
        attr="Map tiles by Stamen Design, CC BY 3.0 — Map data © OpenStreetMap contributors",
        name="Mapa Blanco y Negro"
    ).add_to(mapa_santander)
    folium.TileLayer(
        tiles="https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png",
        attr="Map data © OpenStreetMap contributors, SRTM | Map style: © OpenTopoMap (CC-BY-SA)",
        name="Mapa Satelital"
    ).add_to(mapa_santander)

    # Agregar control de capas
    folium.LayerControl().add_to(mapa_santander)
    
    # Guardar el mapa en un archivo HTML
    mapa_santander.save(f"mapa_santander_{factor_input}_heatmap.html")
    print(f"¡Mapa generado con heatmap! Abre 'mapa_santander_{factor_input}_heatmap.html' en tu navegador para verlo.")

# Función para cargar datos desde un CSV
def cargar_datos_csv(ruta_csv):
    try:
        df_datos = pd.read_csv(ruta_csv)
        return df_datos
    except FileNotFoundError:
        print("Archivo CSV no encontrado.")
        return None

# Función principal para generar el mapa interactivo
if __name__ == "__main__":
    # Cargar los datos desde el archivo CSV
    ruta_csv = "E:\Andrea v4\Trabajos\MED_Juan\Mapas\Santander\datos_sinteticos_santander_con_coordenadas.csv"  # Reemplaza con la ruta de tu archivo CSV
    df_datos = cargar_datos_csv(ruta_csv)
    
    if df_datos is not None:
        # Diccionario con las ubicaciones geográficas de los municipios
        municipios_santander = {
            'Barrancabermeja': [7.05, -73.85],
            'Bucaramanga': [7.12, -73.12],
            'Piedecuesta': [7.04, -73.04]
            # Asegúrate de que todos los municipios en el CSV estén aquí
        }
        
        # Solicitar el factor al usuario
        factor_input = input("Ingresa el factor a analizar (ejemplo: 'Escuela', 'Puentes', 'Placa Huella'): ")
        
        # Verificar si el factor ingresado es válido y generar el mapa con heatmap
        generar_mapa(factor_input, df_datos, municipios_santander)


Advertencia: El municipio 'Girón' no tiene coordenadas definidas. Saltando...
Advertencia: El municipio 'Barichara' no tiene coordenadas definidas. Saltando...
Advertencia: El municipio 'San Gil' no tiene coordenadas definidas. Saltando...
Advertencia: El municipio 'Floridablanca' no tiene coordenadas definidas. Saltando...
Advertencia: El municipio 'Curití' no tiene coordenadas definidas. Saltando...
¡Mapa generado con heatmap! Abre 'mapa_santander_Escuela_heatmap.html' en tu navegador para verlo.
